In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [ ]:
skempiv2 = pd.read_csv('skempi_v2.csv', sep=';')
pdbs = skempiv2['#Pdb'].to_list()
unique = set([x.split('_')[0] for x in pdbs])
print(f'There are {len(unique)} unique PDBs in the dataset.')

In [3]:
orig_size = len(skempiv2)
skempiv2 = skempiv2.dropna(subset=['Affinity_wt_parsed', 'Affinity_mut_parsed'])
print(f'Dropped {orig_size - len(skempiv2)} mutants with NaN values for affinity measurements')

Dropped 287 mutants with NaN values for affinity measurements


## Skempi v2 filtering

In [4]:
### first drop all duplicates
### Note: here we're only keeping the first entry. In some cases it might be better to go over entries manually based on the notes
duplicated_rows = skempiv2[skempiv2.duplicated(subset=['#Pdb', 'Mutation(s)_cleaned'], keep='first')]
skempiv2 = skempiv2.drop_duplicates(subset=['#Pdb', 'Mutation(s)_cleaned'], keep='first')
print(f'Dropped {len(duplicated_rows)} duplicate entries.')


Dropped 884 duplicate entries.


In [5]:
non_interface = ['INT', 'SUR'] # INTerior, SURface
interface = ['COR', 'SUP', 'RIM'] # CORe, SUPport, RIM
filtered_rows = []

for _, row in skempiv2.iterrows():
    mutation_types = row['iMutation_Location(s)']
    for x in mutation_types.split(','):
        if x in interface:
            filtered_rows.append(row)
            break
filtered_skempi = pd.DataFrame(filtered_rows)
print(f'Discarded {len(skempiv2) - len(filtered_skempi)} mutations with only non-interface residues')

Discarded 1029 mutations with only non-interface residues


In [6]:
filtered_skempi['Temperature'] = filtered_skempi['Temperature'].str.split('(').str[0].fillna('298').astype(int)
filtered_skempi['dG_wt'] =  (8.314/4184) * filtered_skempi['Temperature'] * np.log(filtered_skempi['Affinity_wt_parsed'])
filtered_skempi['dG_mut'] =  (8.314/4184) * filtered_skempi['Temperature'] * np.log(filtered_skempi['Affinity_mut_parsed'])
filtered_skempi['ddG'] = -(filtered_skempi['dG_mut'] - filtered_skempi['dG_wt'])

In [7]:
complexes = filtered_skempi.groupby('#Pdb')
filtered_groups = []
discarded = 0
discarded_groups = []
for name, group in complexes:
    if len(group) < 3:
        discarded_groups.append(group)
        discarded += len(group)
    else:
        filtered_groups.append(group)

print(f'Discarded {len(discarded_groups)} complexes/{discarded} mutations with less than 3 mutants')

discarded = 0
discarded_groups = []
filtered_groups2 = []
for group in filtered_groups:
    if group['ddG'].nunique()/len(group) < 0.6:
        discarded_groups.append(group)
        discarded += len(group)
    else:
        filtered_groups2.append(group)

print(f'Discarded {len(discarded_groups)} complexes/{discarded} mutations with more than 40% of the measured ddGs the same value')

Discarded 108 complexes/133 mutations with less than 3 mutants
Discarded 4 complexes/49 mutations with more than 40% of the measured ddGs the same value


In [8]:
from Bio.PDB import PDBParser
directory = '/home/exx/arthur/data/SKEMPI_v2/PDBs'
pdb_list = filtered_skempi['#Pdb'].to_list()
pdb_list = [x.split('_')[0] for x in pdb_list]
pdb_set = set(pdb_list)


In [9]:
unresolved_crystal = []

def find_gaps_in_pdb(pdb_file):
    """
    Find gaps or unresolved residues in a PDB file.

    Args:
        pdb_file (str): Path to the PDB file.

    Returns:
        dict: Dictionary with chain IDs as keys and lists of missing residues as values.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)

    gaps = {}

    for model in structure:
        for chain in model:
            chain_id = chain.id
            residues = [res.id[1] for res in chain if res.id[0] == " "]
            # Residue sequence numbers are in res.id[1]

            # Find missing sequence numbers by comparing consecutive residues
            missing = []
            for i in range(len(residues) - 1):
                if residues[i + 1] != residues[i] + 1:
                    # Add all missing residues between residues[i] and residues[i+1]
                    missing.extend(range(residues[i] + 1, residues[i + 1]))

            if missing:
                gaps[chain_id] = missing

    return gaps

for pdb_name in tqdm(pdb_set):
    # Example usage
    pdb_file = os.path.join(directory, f'{pdb_name}.pdb')
    gaps = find_gaps_in_pdb(pdb_file)

    if gaps:
        for chain, missing in gaps.items():
            unresolved_crystal.append(pdb_name)
            # print(f"Chain {chain}: Missing residues {missing}")


  0%|          | 0/318 [00:00<?, ?it/s]

100%|██████████| 318/318 [00:26<00:00, 11.91it/s]


In [12]:
unresolved_crystal.append('4NKQ') # manual inspection

In [14]:
discarded = 0
discarded_groups = []
filtered_groups3 = []
for group in filtered_groups2:
    if group['#Pdb'].iloc[0].split('_')[0] in unresolved_crystal:
        discarded_groups.append(group)
        discarded += len(group)
    else:
        filtered_groups3.append(group)

print(f'Discarded {len(discarded_groups)} complexes/{discarded} mutations with partially unresolved crystal structure')

filtered_skempi = pd.concat(filtered_groups3)
print(f'Filtered from SKEMPIv2: {len(filtered_groups3)} complexes/{sum([len(group) for group in filtered_groups3])} mutants.')
print(f'The resulting datset has {len(filtered_skempi)} mutants.')

Discarded 8 complexes/162 mutations with partially unresolved crystal structure
Filtered from SKEMPIv2: 201 complexes/4541 mutants.
The resulting datset has 4541 mutants.


In [15]:
filtered_skempi.to_csv('filtered_skempi.csv')